### Creamos el Dataframe

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

ruta = '/home/luisa/Downloads/df_estados_bank.csv'


df = pd.read_csv(ruta, sep=',')



### Creamos una nueva copia y hacemos el EDA

In [ ]:
dfN = df.copy()

# Mapeo para la variable Género

mapeo_genero = {'Female': 0, 'Male': 1}
dfN['Gender'] = dfN['Gender'].map(mapeo_genero)
dfN

# Mapeo para la variable Estado
mapeo_estado = {'Texas': 0, 'California': 1, 'Alabama': 2}
dfN['Geography'] = dfN['Geography'].map(mapeo_estado) # Imprimir el DataFrame actualizado print(dfN7)
dfN

# *** Ya que tenemos claro que NO nos sirve de nada esa columna, la borramos y creamos un nuevo 'DFN3'
dfN = dfN.drop(['RowNumber'], axis=1)
dfN = dfN.drop(['Unnamed: 0'], axis=1)
dfN


# Quitamos la columna de la Identificación porque creemos que no nos aporta nada para la predicción (CustomerId)
dfN = dfN.drop(['CustomerId'], axis=1)
dfN

# Quitamos la columna del apellido (Surname)
dfN = dfN.drop(['Surname'], axis=1)
dfN

# Reemplazo los nulos con la mediana
dfN['Age'].fillna(dfN['Age'].median(), inplace=True)
dfN['EstimatedSalary'].fillna(dfN['EstimatedSalary'].median(), inplace=True)
dfN

#Mapeo , se eliminan outliers
columnas_outliers = ["Age",]

for columna in columnas_outliers:
    Q1 = np.quantile(dfN[columna], 0.25)
    Q3 = np.quantile(dfN[columna], 0.75)
    IQR = Q3 - Q1

    lower_limit = Q1 - 1.5 * IQR
    upper_limit = Q3 + 1.5 * IQR

dfN[columna] = np.clip(dfN[columna], lower_limit, upper_limit)

#En este caso eliminaremos la variable "Exited" para probar nuestro df con las variables independientes
#Creamos la variable target a parte
dfN_Target = dfN['Exited']
dfN = dfN.drop('Exited', axis=1)
dfN

,CreditScore,Gender,Balance,NumOfProducts,HasCrCard,IsActiveMember,Age,Geography,EstimatedSalary,Tenure
0,619,0,0.00,1,1,1,42.0,0,101348.88,2
1,608,0,83807.86,1,0,1,41.0,1,112542.58,1
2,502,0,159660.80,3,1,0,42.0,0,113931.57,8
3,699,0,0.00,2,0,0,39.0,0,93826.63,1
4,850,0,125510.82,1,1,1,43.0,1,79084.10,2
...,...,...,...,...,...,...,...,...,...,...
9995,771,1,0.00,2,1,0,39.0,0,96270.64,5
9996,516,1,57369.61,1,1,1,35.0,0,101699.77,10
9997,709,0,0.00,1,0,1,37.0,0,42085.58,7
9998,772,1,75075.31,2,1,0,42.0,2,92888.52,3


### Usamos Regresión logística

#### Primero haremos una regresión logística simple

* La elegimos ya que se utiliza para predecir una variable categórica binaria como es nuestro caso.
* Es una extensión de la regresión lineal, en la cual se utiliza la función logística para modelar
    la probabilidad de que un evento ocurra (probabilidad de clase 1) a partir de una combinación
    lineal de las variables independientes


In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix


# importamos train_test_split que es para separar los datos

from sklearn.model_selection import train_test_split

X = dfN # ==> nuestro dataframe
y = dfN_Target # ==> Target

#Con test_size definimos el porcentaje de datos a entrenar

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)


#Definimos el modelo de regresión logística

clf = LogisticRegression()

#Entrenamos el modelo con el conjunto de entrenamiento

clf.fit(X_train, y_train)

#Realizamos las predicciones con el conjunto de prueba

y_pred = clf.predict(X_test)

#Imprimimos la precisión del modelo

print("Precisión: {:2f}%".format(clf.score(X_test,y_test)*100))


# Haremos una matriz de confusión para calcular las metricas respectivas y ver el rendimiento del modelo

matriz = confusion_matrix(y_test , y_pred)
print(matriz)

Precisión: 84.200000%
[[1684    0]
 [ 316    0]]


#### Luego haremos una regresión logística con StandardScaler

 *Al estandarizar las características, todas tendrán una media de cero y una desviación estándar de uno, lo que asegura que las características estén en la misma escala y evita que una característica con una magnitud mayor tenga un impacto dominante.*

In [ ]:
from sklearn.preprocessing import StandardScaler


X = dfN # ==> nuestro dataframe
y = dfN_Target # ==> nuesta target

#Con test_size definimos el porcentaje de datos a entrenar

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)


# escalamos los  datos para llevarlos a una escala de mismas magnitudes
escalar = StandardScaler()

#con fit_transform nos devuelve los datos ya escalados

X_train=escalar.fit_transform(X_train)
X_test=escalar.fit_transform(X_test)

#Definimos el modelo de regresión logística

clf = LogisticRegression ()

#Entrenamos el modelo

clf.fit(X_train, y_train)

#Realizamos la predicción con la instrucción "Predict"

y_pred = clf.predict(X_test)

#Imprimimos la precisión del modelo

print("Precisión: {:2f}%".format(clf.score(X_test,y_test)*100))

# Haremos una matriz de confusión para calcular las metricas respectivas y ver el rendimiento del modelo

matriz = confusion_matrix(y_test , y_pred)
print(matriz)


Precisión: 84.300000%
[[1636   27]
 [ 287   50]]


### Random Forest

* Es una técnica que combina múltiples árboles de decisión individuales para obtener predicciones más precisas y robustas.
* Random Forest es efectivo en conjuntos de datos complejos que contienen múltiples características y relaciones no lineales como es nuestro caso en nuestro dataset.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#Creamos los datos de entrenamiento

X, y = dfN, dfN_Target

#Creamos una instancia con RandomForestClassifier
#n_estimators = número de árboles

clf = RandomForestClassifier(n_estimators = 100, max_depth=3)

#Entrenamos el modelo con los datos de entrenamiento

clf = clf.fit(X,y)

#Mostramos la puntuación del modelo

print("Puntuación:", clf.score(X,y))

#Realizamos una predicción

# La puntuación va del 0 al 1

Puntuación: 0.8407
